I have never really learned or worked on this before and it was very new for me. I tried my best to understand this and come up with the best code possible but given the time constraints this was what I could come up with. I will definitely try to learn more about this as this assignment really got me interested. i have basically tried to create a bot which helps like a travel guide. This is a very basic execution of my idea.

In [ ]:
!pip install sentence_transformers
!pip install faiss-cpu
import torch
from sentence_transformers import SentenceTransformer, util
import faiss
from transformers import T5ForConditionalGeneration, T5Tokenizer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

I installed and imported all the required libraries

In [ ]:

# Loaded a pre-trained sentence transformer model
embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

# took this very basic dataset
passages = [
    "Paris, the City of Light, beckons with its romantic ambiance and iconic landmarks such as the Eiffel Tower and Notre-Dame Cathedral.",
    "The Great Wall of China, a UNESCO World Heritage Site, offers breathtaking views and a glimpse into China's rich history.",
    "Tokyo, a bustling metropolis blending tradition and innovation, promises endless exploration from the historic Senso-ji Temple to the futuristic district of Akihabara.",
    "The Maldives, a tropical paradise of turquoise waters and pristine beaches, is the ultimate destination for luxury and relaxation.",
    "Nepal's Annapurna Circuit offers trekkers a journey through stunning landscapes, remote villages, and the hospitality of the Himalayan people.",
    "Iceland's Golden Circle route features dramatic landscapes including waterfalls, geysers, and the famous Blue Lagoon, making it a nature lover's dream.",
    "Bali, known as the Island of the Gods, enchants visitors with its lush landscapes, spiritual temples, and vibrant arts scene.",
    "Machu Picchu, the lost city of the Incas nestled in the Andes Mountains, is a testament to ancient engineering and breathtaking natural beauty.",
]


passage_embeddings = embedder.encode(passages, convert_to_tensor=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

I used a pre trained model "roberta-base-nli-stsb-mean-tokens" for sentence embeddings basically to convert the sentences into numerical vectors.

Created a passage with information about some destinations.

In [ ]:

# Created FAISS index
index = faiss.IndexFlatL2(passage_embeddings.shape[1])
index.add(passage_embeddings.cpu().detach().numpy())

# Loaded a pre-trained T5 model and tokenizer
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded a pre trained T5 model and its tokenizer

In [ ]:

def retrieve_passages(query, top_k=3):
    query_embedding = embedder.encode([query], convert_to_tensor=True)
    query_embedding = query_embedding.cpu().detach().numpy()

    # Computed cosine similarity between query and passages
    passage_similarities = util.pytorch_cos_sim(torch.tensor(query_embedding), torch.tensor(passage_embeddings)).cpu().detach().numpy()[0]

    # Sorted passages based on similarity
    sorted_indices = sorted(range(len(passage_similarities)), key=lambda i: passage_similarities[i], reverse=True)
    top_indices = sorted_indices[:top_k]

    # Retrieved the top-k passages
    results = [passages[idx] for idx in top_indices]
    return results


Main idea of code is: It takes up the query of user and finds similarity between Query and each line of passage. I don't have a great knowledge about how attention mechanism can be used so came up with idea. After finding the similarity it returns top 3 lines of passage which have higher similarity with query

In [ ]:

def chatbot_response(query):
    retrieved_passages = retrieve_passages(query)
    combined_input = " ".join(retrieved_passages)

    # Prepared the input for the T5 model
    t5_input_text = f"query: {query} context: {combined_input}"
    input_ids = t5_tokenizer.encode(t5_input_text, return_tensors='pt')

    # Fine-tuned T5 model to generate more personalized responses
    fine_tuned_model = t5_model
    fine_tuned_model.train()  # Enable training mode for fine-tuning
    fine_tuned_model.resize_token_embeddings(len(t5_tokenizer))  # Resize token embeddings

    # Generated the response using fine-tuned T5 model
    outputs = fine_tuned_model.generate(input_ids)
    response = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response


This part is basically the core of the chatbot that formulates responses based on user queries and retrieved passages. The output from retrieve_passages function is joined together as a single line and this along with query is sent as single input for T5 model. Response is generated using fine tuned T5 model and given as output

In [ ]:


query = "Where can I experience waterfalls, geysers, and the famous Blue Lagoon"
response = chatbot_response(query)
print(f"User: {query}")
print(f"Chatbot: {response}")


<ipython-input-4-af6379d6ddae>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  passage_similarities = util.pytorch_cos_sim(torch.tensor(query_embedding), torch.tensor(passage_embeddings)).cpu().detach().numpy()[0]


User: Where can I experience waterfalls, geysers, and the famous Blue Lagoon
Chatbot: Iceland's Golden Circle route


This is testing part where I asked some question to check what answer is being given.

# Analysis

This model is not so apt and there is lot of scope for improvement.

I tried passing the query 'I'm planning an adventurous trip, any recommendations?' and it replied 'Nepal's Annapurna Circuit', totally messing up but it works good on clearly stated queries like 'I'm planning a trip to Iceland, any recommendations?' or 'Where can I experience waterfalls, geysers, and the famous Blue Lagoon?'.

<br>
<br>

## Ways to improve

* **Model Selection** -
    The most basic way to improve is to go for better models like BERT or GPT.
    I did try working with Open AI but didn't really work well with my code so went ahead with T5

* **Fine tuning** -
    The another method I could have used is fine tuning model. But due to less time I went ahead with this.

* **Dataset** -
    I have taken a very minimal passage to select my answer from. Increasing this can further increase the accuracy of this.

This lags many features which an actual chatbot has and can be included in this and this will better the user experience.




